In [1]:
dataset_path = '../bin/multirc_dataset.hf'
mpnet_name = '/tiedaar/summary-longformer-wording'
bleurt_name = 'vaiibhavgupta/finetuned-bleurt-large'

In [ ]:
!pip install transformers
!pip install datasets

from transformers import pipeline, AutoTokenizer
from datasets import DatasetDict
import pandas as pd
from tqdm import tqdm
import torch

tqdm.pandas()

In [4]:
torch.cuda.is_available()

False

In [5]:
mpnet_pipe = pipeline('text-classification', model='tiedaar/short-answer-classification', device=-1)
bleurt_pipe = pipeline('text-classification', model="vaiibhavgupta/finetuned-bleurt-large", device=-1)
bleurt_tokenizer = AutoTokenizer.from_pretrained("vaiibhavgupta/finetuned-bleurt-large")

In [6]:
ds = DatasetDict.load_from_disk(dataset_path)

In [7]:
test_df = ds['test'].to_pandas()

In [8]:
df = pd.read_csv('../data/multirc.csv')
column_index = list(df.columns)
column_index[0] = 'index'
df.columns = column_index
test_df = df.merge(test_df, on='index', how='right')
test_df['labels'] = test_df['labels'].progress_apply(lambda x: 'correct_answer' if x==1 else 'incorrect_answer')

100%|██████████| 3962/3962 [00:00<00:00, 988274.31it/s]


,index,question_text,source_text_clean,gpt_answers,source_id,source_text,answer_text,is_correct,text,labels
0,561,"Thanks to a grant, which shelter will now be a...",Legal assistance for battered women is hard to...,\nAnswer: The Women's Haven of Tarrant County ...,News/NYT/masc-20020731-nyt-4.txt,<b>Sent 1: </b>Legal assistance for battered w...,The Women's Haven of Tarrant County,True,The Women's Haven of Tarrant County</s>\nAnswe...,correct_answer
1,562,"Thanks to a grant, which shelter will now be a...",Legal assistance for battered women is hard to...,\nAnswer: The Women's Haven of Tarrant County ...,News/NYT/masc-20020731-nyt-4.txt,<b>Sent 1: </b>Legal assistance for battered w...,Tarrant county shelters,False,Tarrant county shelters</s>\nAnswer: The Women...,incorrect_answer
2,563,"Thanks to a grant, which shelter will now be a...",Legal assistance for battered women is hard to...,\nAnswer: The Women's Haven of Tarrant County ...,News/NYT/masc-20020731-nyt-4.txt,<b>Sent 1: </b>Legal assistance for battered w...,Female's Safe House of Haven County,False,Female's Safe House of Haven County</s>\nAnswe...,incorrect_answer
3,564,Women who are not divorced can't obtain child ...,Legal assistance for battered women is hard to...,\nAnswer: Texas,News/NYT/masc-20020731-nyt-4.txt,<b>Sent 1: </b>Legal assistance for battered w...,California,False,California</s>\nAnswer: Texas,incorrect_answer
4,565,Women who are not divorced can't obtain child ...,Legal assistance for battered women is hard to...,\nAnswer: Texas,News/NYT/masc-20020731-nyt-4.txt,<b>Sent 1: </b>Legal assistance for battered w...,Texas,True,Texas</s>\nAnswer: Texas,correct_answer
...,...,...,...,...,...,...,...,...,...,...
3957,27067,What created the Sun and moon?,How would the universe look without gravity?\n...,Answer: Gravity caused dust and gas particles ...,News/NYT/masc-20020731-nyt-4.txt,<b>Sent 1: </b>How would the universe look wit...,Gravity,True,Gravity</s>Answer: Gravity caused dust and gas...,correct_answer
3958,27068,What created the Sun and moon?,How would the universe look without gravity?\n...,Answer: Gravity caused dust and gas particles ...,News/NYT/masc-20020731-nyt-4.txt,<b>Sent 1: </b>How would the universe look wit...,"Gravity formed the smallest moons, it also for...",True,"Gravity formed the smallest moons, it also for...",correct_answer
3959,27069,What created the Sun and moon?,How would the universe look without gravity?\n...,Answer: Gravity caused dust and gas particles ...,News/NYT/masc-20020731-nyt-4.txt,<b>Sent 1: </b>How would the universe look wit...,Galxies,False,Galxies</s>Answer: Gravity caused dust and gas...,incorrect_answer
3960,27070,What created the Sun and moon?,How would the universe look without gravity?\n...,Answer: Gravity caused dust and gas particles ...,News/NYT/masc-20020731-nyt-4.txt,<b>Sent 1: </b>How would the universe look wit...,Stars,False,Stars</s>Answer: Gravity caused dust and gas p...,incorrect_answer


In [14]:
def get_mpnet(row):

    text = candidate + '</s>' + reference
    res = mpnet_pipe(text)[0]['label']
    return res
    
test_df['mpnet_preds'] = test_df.progress_apply(lambda row: get_mpnet(row), axis=1)

100%|██████████| 3962/3962 [16:56<00:00,  3.90it/s]


In [16]:
def get_bleurt(row):
    candidate = row['answer_text']
    reference = row['gpt_answers']
    text = candidate + bleurt_tokenizer.sep_token + reference
    score = bleurt_pipe(text)[0]['score']
    if score > 0.7:
        return 'correct_answer'
    else:
        return 'incorrect_answer'

test_df['bleurt_preds'] = test_df.progress_apply(lambda row: get_bleurt(row), axis=1)

100%|██████████| 3962/3962 [44:52<00:00,  1.47it/s]  


In [17]:
test_df

,index,question_text,source_text_clean,gpt_answers,source_id,source_text,answer_text,is_correct,text,labels,mpnet_preds,bleurt_preds
0,561,"Thanks to a grant, which shelter will now be a...",Legal assistance for battered women is hard to...,\nAnswer: The Women's Haven of Tarrant County ...,News/NYT/masc-20020731-nyt-4.txt,<b>Sent 1: </b>Legal assistance for battered w...,The Women's Haven of Tarrant County,True,The Women's Haven of Tarrant County</s>\nAnswe...,correct_answer,correct_answer,correct_answer
1,562,"Thanks to a grant, which shelter will now be a...",Legal assistance for battered women is hard to...,\nAnswer: The Women's Haven of Tarrant County ...,News/NYT/masc-20020731-nyt-4.txt,<b>Sent 1: </b>Legal assistance for battered w...,Tarrant county shelters,False,Tarrant county shelters</s>\nAnswer: The Women...,incorrect_answer,incorrect_answer,incorrect_answer
2,563,"Thanks to a grant, which shelter will now be a...",Legal assistance for battered women is hard to...,\nAnswer: The Women's Haven of Tarrant County ...,News/NYT/masc-20020731-nyt-4.txt,<b>Sent 1: </b>Legal assistance for battered w...,Female's Safe House of Haven County,False,Female's Safe House of Haven County</s>\nAnswe...,incorrect_answer,incorrect_answer,incorrect_answer
3,564,Women who are not divorced can't obtain child ...,Legal assistance for battered women is hard to...,\nAnswer: Texas,News/NYT/masc-20020731-nyt-4.txt,<b>Sent 1: </b>Legal assistance for battered w...,California,False,California</s>\nAnswer: Texas,incorrect_answer,correct_answer,incorrect_answer
4,565,Women who are not divorced can't obtain child ...,Legal assistance for battered women is hard to...,\nAnswer: Texas,News/NYT/masc-20020731-nyt-4.txt,<b>Sent 1: </b>Legal assistance for battered w...,Texas,True,Texas</s>\nAnswer: Texas,correct_answer,correct_answer,correct_answer
...,...,...,...,...,...,...,...,...,...,...,...,...
3957,27067,What created the Sun and moon?,How would the universe look without gravity?\n...,Answer: Gravity caused dust and gas particles ...,News/NYT/masc-20020731-nyt-4.txt,<b>Sent 1: </b>How would the universe look wit...,Gravity,True,Gravity</s>Answer: Gravity caused dust and gas...,correct_answer,correct_answer,correct_answer
3958,27068,What created the Sun and moon?,How would the universe look without gravity?\n...,Answer: Gravity caused dust and gas particles ...,News/NYT/masc-20020731-nyt-4.txt,<b>Sent 1: </b>How would the universe look wit...,"Gravity formed the smallest moons, it also for...",True,"Gravity formed the smallest moons, it also for...",correct_answer,correct_answer,correct_answer
3959,27069,What created the Sun and moon?,How would the universe look without gravity?\n...,Answer: Gravity caused dust and gas particles ...,News/NYT/masc-20020731-nyt-4.txt,<b>Sent 1: </b>How would the universe look wit...,Galxies,False,Galxies</s>Answer: Gravity caused dust and gas...,incorrect_answer,incorrect_answer,incorrect_answer
3960,27070,What created the Sun and moon?,How would the universe look without gravity?\n...,Answer: Gravity caused dust and gas particles ...,News/NYT/masc-20020731-nyt-4.txt,<b>Sent 1: </b>How would the universe look wit...,Stars,False,Stars</s>Answer: Gravity caused dust and gas p...,incorrect_answer,incorrect_answer,incorrect_answer


In [35]:
temp_df = test_df.loc[(test_df['mpnet_preds'] == 'incorrect_answer') & (test_df['bleurt_preds']=='correct_answer')]
temp_df['labels'].value_counts()

labels
incorrect_answer    99
correct_answer      69
Name: count, dtype: int64

In [42]:
len(test_df[test_df['labels']=='incorrect_answer'])

2240